<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv4/blob/main/RBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/insertion_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/removal_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/adverserial_attacks_functions.py'
]

In [2]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 8.43MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 5.32MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 5.40MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 7.49MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 3.92MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from adverserial_attacks_functions import *


torch.manual_seed(0)



cuda
cuda


In [4]:
# Open the .pkl file
with open('/content/insertion_array.pkl', 'rb') as f:
    # Load the object
    insertion_array = pickle.load(f)

# Close the file
f.close()

insertion_array = torch.tensor(insertion_array).to(device)
print(len(insertion_array))

# Open the .pkl file
with open('/content/removal_array.pkl', 'rb') as f:
    # Load the object
    removal_array = pickle.load(f)

# Close the file
f.close()

removal_array = torch.tensor(removal_array).to(device)
print(len(removal_array))


10000
10000


In [5]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [6]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [7]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [8]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [9]:
# Create an instance of your model
model_AT_rFGSM_weightedLoss = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM_weightedLoss.load_state_dict(torch.load('model_AT_rFGSM_weightedLoss.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [15]:
from sklearn.cluster import KMeans

# Function to initialize centers and sigmas separately for benign and malware samples using KMeans clustering
def initialize_centers_sigmas_separate(data_loader, num_centers_per_class, removal_array = removal_array, non_removal_features = False):
    centers_benign = []
    centers_malware = []
    non_removal_mask = torch.logical_not(removal_array).to('cpu')

    # Collect data into a single tensor
    all_data = torch.cat([batch for batch, _ in data_loader], dim=0)
    if non_removal_features:
        all_data = all_data[:,non_removal_mask]

    all_data = all_data.numpy()
    # Collect labels into a single tensor
    all_labels = torch.cat([labels for _, labels in data_loader], dim=0)
    all_labels = all_labels.numpy()

    # Separate benign and malware samples
    benign_indices = np.where(all_labels == 0)[0]
    malware_indices = np.where(all_labels == 1)[0]

    # Clustering for benign samples
    benigns = all_data[benign_indices]
    if non_removal_features:
        subset_benigns = benigns
    else:
        subset_benigns = benigns[:20000]

    kmeans_benign = KMeans(n_clusters=num_centers_per_class, init='k-means++', n_init='auto')
    kmeans_benign.fit(subset_benigns)
    centers_selected_benign = kmeans_benign.cluster_centers_

    # Clustering for malware samples
    kmeans_malware = KMeans(n_clusters=num_centers_per_class, init='k-means++', n_init='auto')
    kmeans_malware.fit(all_data[malware_indices])
    centers_selected_malware = kmeans_malware.cluster_centers_

    # Combine selected centers
    all_centers = np.concatenate([centers_selected_benign, centers_selected_malware], axis=0)

    # Calculate sigma based on the average distance between centers
    total_distance = 0.0
    num_pairs = 0

    # Calculate pairwise distances between centers for benign samples
    for i in range(len(centers_selected_benign)):
        for j in range(i + 1, len(centers_selected_benign)):
            distance_ij = np.sqrt(((centers_selected_benign[i] - centers_selected_benign[j]) ** 2).sum())
            total_distance += distance_ij
            num_pairs += 1

    # Calculate pairwise distances between centers for malware samples
    for i in range(len(centers_selected_malware)):
        for j in range(i + 1, len(centers_selected_malware)):
            distance_ij = np.sqrt(((centers_selected_malware[i] - centers_selected_malware[j]) ** 2).sum())
            total_distance += distance_ij
            num_pairs += 1

    # Calculate mean sigma

    sigma = total_distance / num_pairs

    return all_centers, sigma


In [11]:
all_centers, sigma = initialize_centers_sigmas_separate(train_loader, 500)

In [12]:
sigma

6.325346210357184

In [13]:
all_centers.shape

(1000, 10000)

In [14]:
# Save the array to a file
np.save('all_centers.npy', all_centers)

In [16]:
all_centers_non_removal_features, sigma_non_removal_features = initialize_centers_sigmas_separate(train_loader, 500, removal_array = removal_array, non_removal_features = True)

In [17]:
print(all_centers_non_removal_features.shape)
print(sigma_non_removal_features)

(1000, 1144)
3.967844595954441


In [29]:
all_centers_non_removal_features[0][:20]

array([ 4.76190476e-02, -5.55111512e-17,  4.76190476e-02, -4.16333634e-17,
        4.76190476e-02,  0.00000000e+00, -1.38777878e-17, -6.93889390e-18,
        9.52380952e-02, -5.55111512e-17,  4.76190476e-02, -2.77555756e-17,
       -1.38777878e-17,  8.09523810e-01, -6.93889390e-18,  0.00000000e+00,
        0.00000000e+00,  2.08166817e-17,  1.42857143e-01,  1.04083409e-17])

In [18]:
# Save the array to a file
np.save('all_centers_non_removal_features.npy', all_centers_non_removal_features)

In [11]:
# Load the array from the file
all_centers = np.load('all_centers.npy')

sigma = 6.281732838681898

In [12]:
# Move all_centers back to GPU
all_centers = torch.tensor(all_centers, device=device)

In [20]:
all_centers_non_removal_features = torch.tensor(all_centers_non_removal_features, device=device)

<ipython-input-20-b585ae409818>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_centers_non_removal_features = torch.tensor(all_centers_non_removal_features, device=device)


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from sklearn.cluster import KMeans

# Define the RBF model
class RBFModel(nn.Module):
    def __init__(self, hidden_dim, output_dim, init_centers, init_sigmas):
        super(RBFModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.centers = nn.Parameter(torch.Tensor(init_centers))
        self.sigmas = nn.Parameter(torch.Tensor(init_sigmas))

        # Linear layer for output
        self.linear = nn.Linear(hidden_dim, output_dim)

    def gaussian(self, x, c, sigma):
        return torch.exp(-torch.sum((x[:, None, :] - c) ** 2, dim=-1) / (2 * sigma ** 2))

    def forward(self, x):
        radial_out = self.gaussian(x, self.centers, self.sigmas)
        output = self.linear(radial_out.to(torch.float32))
        return output

# Function to evaluate the model on a dataset
def evaluate_model(model, data_loader, criterion, removal_array = removal_array, non_removal_features = False):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    non_removal_mask = torch.logical_not(removal_array).to('cpu')

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            if non_removal_features:
                batch_x, batch_y = batch_x[:,non_removal_mask].to(torch.float32).to(device), batch_y.to(device)  # Move data to GPU
            else:
                batch_x, batch_y = batch_x.to(torch.float32).to(device), batch_y.to(device)  # Move data to GPU
            output = model(batch_x)
            loss = criterion(output, batch_y.squeeze().to(torch.long))
            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            correct_predictions += (predicted == batch_y.squeeze()).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / len(data_loader.dataset)

    return avg_loss, accuracy


In [16]:
# Run this to release GPU memory
torch.cuda.empty_cache()

In [ ]:
non_removal_features = True
non_removal_mask = torch.logical_not(removal_array).to('cpu')

model = RBFModel(1000, 2, all_centers_non_removal_features, [sigma_non_removal_features])
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)  # Decrease lr by a factor of 0.95 every 1 epoch

# Training loop
num_epochs = 30
best_val_accuracy = 0.0
best_model_state_dict = None

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0

    # Print the current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print('************************************')
    print(f'Current Learning Rate: {current_lr}')

    for i, (batch_x, batch_y) in enumerate(train_loader):
        # Remove features based on removal_array
        if non_removal_features:
            batch_x = batch_x[:, non_removal_mask].to(torch.float32).to(device)  # Select features
        else:
            batch_x = batch_x.to(torch.float32).to(device)  # Move all features to GPU
        batch_y = batch_y.to(device)  # Move labels to GPU

        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y.squeeze().to(torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct_predictions += (predicted == batch_y.squeeze()).sum().item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss}, Train Accuracy: {accuracy}')

    # Step the scheduler
    scheduler.step()

    # Evaluate on validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, removal_array = removal_array, non_removal_features=non_removal_features)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        print(f"New best validation accuracy: {best_val_accuracy}")
        # Save the current best model state dict
        best_model_state_dict = model.state_dict()

# Save the best model
if best_model_state_dict is not None:
    torch.save(best_model_state_dict, 'best_model.pth')
    print("Best model saved with validation accuracy:", best_val_accuracy)
else:
    print("No improvement in validation accuracy.")


************************************
Current Learning Rate: 0.001
Epoch [1/30], Step [10/113], Loss: 0.48220548033714294
Epoch [1/30], Step [20/113], Loss: 0.2858605682849884
Epoch [1/30], Step [30/113], Loss: 0.2665095031261444
Epoch [1/30], Step [40/113], Loss: 0.21803133189678192
Epoch [1/30], Step [50/113], Loss: 0.22780616581439972
Epoch [1/30], Step [60/113], Loss: 0.16350334882736206
Epoch [1/30], Step [70/113], Loss: 0.31015217304229736
Epoch [1/30], Step [80/113], Loss: 0.20080940425395966
Epoch [1/30], Step [90/113], Loss: 0.1882152557373047
Epoch [1/30], Step [100/113], Loss: 0.19890359044075012
Epoch [1/30], Step [110/113], Loss: 0.17353910207748413
Epoch [1/30], Train Loss: 0.2583295907330724, Train Accuracy: 0.8975699891922044
Epoch [1/30], Validation Loss: 0.16741640140351496, Validation Accuracy: 0.9479188454298264
New best validation accuracy: 0.9479188454298264
************************************
Current Learning Rate: 0.00095
Epoch [2/30], Step [10/113], Loss: 0.181

In [36]:
# Re-instantiate the model
model = RBFModel(10000, 1000, 2, all_centers, [sigma])

# Load the saved model state dictionary
model_state_dict = torch.load('best_model.pth')

# Load the model state dictionary into the model
model.load_state_dict(model_state_dict)

model = model.to(device)

# Ensure the model is in evaluation mode
model.eval()


RBFModel(
  (linear): Linear(in_features=1000, out_features=2, bias=True)
)

In [37]:
for i, (batch_x, batch_y) in enumerate(train_loader):
        batch_x, batch_y = batch_x.to(torch.float32).to(device), batch_y.to(device)
        output = model(batch_x)
        break

In [39]:
batch_y

tensor([[0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], device='cuda:0', dtype=torch.int8)

In [38]:
output

tensor([[ 3.5949, -3.7890],
        [ 2.3963, -2.5447],
        [ 4.7550, -4.9008],
        [-2.0959,  1.8883],
        [ 1.0827, -1.2903],
        [ 4.2781, -4.4563],
        [ 1.8530, -2.1296],
        [ 2.4713, -2.7093],
        [ 3.5167, -3.6290],
        [ 3.1558, -3.3632],
        [ 2.7086, -2.9140],
        [ 4.2223, -4.3793],
        [ 2.4756, -2.7242],
        [ 5.2013, -5.3457],
        [ 3.3582, -3.5315],
        [ 4.8625, -5.0373],
        [ 2.8663, -3.1130],
        [ 2.2492, -2.4581],
        [ 2.9067, -3.1698],
        [ 3.5118, -3.6695],
        [ 3.2226, -3.4617],
        [ 1.3615, -1.5182],
        [ 2.4652, -2.7245],
        [ 3.8473, -4.0772],
        [ 2.7479, -2.9086],
        [ 4.8274, -4.9394],
        [ 4.1073, -4.3304],
        [ 2.5318, -2.7993],
        [ 3.7883, -4.0139],
        [ 3.2683, -3.5091],
        [ 2.5497, -2.8065],
        [ 3.9403, -4.1609]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [40]:
# After training, you can extract the trained parameters
trained_centers = model.centers.detach().cpu().numpy()
trained_sigmas = model.sigmas.detach().cpu().numpy()

# Print or use the trained parameters
print("Trained Centers:", trained_centers)
print("Trained Sigmas:", trained_sigmas)

Trained Centers: [[ 6.82626901e-01 -1.57842796e-01 -9.10633264e-01 ... -1.35908069e-03
  -7.18771835e-06 -3.75721239e-07]
 [-6.69717064e-02 -1.14904760e-01 -9.74693168e-01 ... -7.95601256e-04
  -1.61720848e-06  5.57291111e-08]
 [ 1.13970780e-01  1.01621359e-02 -8.41641331e-01 ... -1.29463659e-03
  -5.19032333e-06 -9.45904902e-07]
 ...
 [ 3.28155071e-02 -8.84417520e-02  1.41652337e+00 ...  8.22687592e-04
  -1.71252737e-04  1.12640569e-07]
 [-5.62768074e-02 -8.98881165e-02 -9.67118275e-01 ... -7.32456894e-04
  -8.73512269e-06 -3.86629888e-08]
 [ 1.01287085e+00  2.35785213e-01  1.07589317e+00 ...  1.32226698e-03
  -1.63619668e-04  2.84134646e-07]]
Trained Sigmas: [4.9973483]


In [44]:
trained_centers.shape

(1000, 10000)

In [43]:
trained_centers[0][:100]

array([ 6.82626901e-01, -1.57842796e-01, -9.10633264e-01,  2.37577198e-01,
       -6.26838652e-02,  6.70036206e-01, -1.60009342e-01, -2.23739003e-01,
        1.18476455e-01, -3.87529193e-01,  2.73201868e-02, -4.53082282e-01,
       -4.94938135e-01,  9.96039044e-01,  4.91933296e-02,  8.45646006e-02,
       -3.00290185e-01, -6.18974304e-02,  8.16441501e-01,  7.67651765e-01,
        4.17308083e-01,  3.52254245e-02,  1.71979274e-01,  1.04967470e-01,
        1.13975330e-01,  1.22958523e+00, -6.97449781e-02, -3.97829565e-01,
        2.14242215e-01,  4.87425539e-01, -2.96133903e-01, -3.67946295e-02,
        7.64374631e-02, -1.10759236e-01,  9.25623384e-02, -2.34933596e-01,
        4.64434705e-01,  1.00638761e+00, -5.27856739e-01,  8.17454471e-02,
       -2.76793184e-02,  1.77565128e-01, -3.23893320e-01, -5.36100208e-01,
       -2.74831020e-01, -2.64936380e-01, -2.19044896e-01, -2.59677026e-01,
       -3.90212805e-01,  2.23359263e-01,  2.52186492e-01, -2.46078962e-01,
        7.84815601e-01, -

In [46]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
